In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
import concurrent.futures

# Read recipe inputs
protein_compatibility_particles = dataiku.Folder("PIojoFlG")
protein_compatibility_particles_info = protein_compatibility_particles.get_info()
particles_from_PS20_degradation = dataiku.Folder("59PiMVgQ")
particles_from_PS20_degradation_info = particles_from_PS20_degradation.get_info()
contact_with_bag_surface = dataiku.Folder("AZvdljVp")
contact_with_bag_surface_info = contact_with_bag_surface.get_info()
protein_stir = dataiku.Folder("Y8jS78ta")
protein_stir_info = protein_stir.get_info()
protein_vortex = dataiku.Folder("A0nzUMQ2")
protein_vortex_info = protein_vortex.get_info()
siliconOil_droplets_from_PFS = dataiku.Folder("Oay7P2WG")
siliconOil_droplets_from_PFS_info = siliconOil_droplets_from_PFS.get_info()
silicon_Catheters = dataiku.Folder("mbeOHgg1")
silicon_Catheters_info = silicon_Catheters.get_info()




# Write recipe outputs
svp_clusters = dataiku.Folder("hAbB1mJR")
svp_clusters_info = svp_clusters.get_info()
svp_clusters_info

In [0]:
#Import all the necessary packages
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
import cv2
import io
import time
from keras.models import load_model, Model
from keras.preprocessing import image
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
import matplotlib.image as mpimg
from PIL import Image, ImageStat
from functools import reduce
import os, shutil, glob, os.path
from PIL import Image as pil_image
image.LOAD_TRUNCATED_IMAGES = True
model = VGG16(weights='imagenet', include_top=False)

map_folder_labels = {    'Y8jS78ta': 'p4',
                         'A0nzUMQ2': 'p5',
                         'Oay7P2WG': 's1',
                         'mbeOHgg1': 's2',
                         'PIojoFlG': 'p1',
                         '59PiMVgQ': 'p3',
                         'AZvdljVp': 'p2'}

In [0]:
# Read csv filE
def get_image_data(name_folder, label_name):
    conn_folder = dataiku.Folder(name_folder)
    all_files_paths = conn_folder.list_paths_in_partition()
    csv_paths = [file_path for file_path in all_files_paths if file_path.endswith('.csv')]
    complete_df = pd.DataFrame()
    for path in csv_paths:
        lis_records = []
        with conn_folder.get_download_stream(path) as f:
            byte_data = f.readlines()
            for row in byte_data:
                #print(row)
                lis_records.append(row.decode('utf-8', 'ignore').replace('\r\n','').split(',')[1:])
        filename = lis_records[10][0]
        #print(filename)
        folder_path = filename.split("\\")[-2]
        #print(folder_path)
        df = pd.DataFrame(lis_records[29:], columns = lis_records[28])
        df['Folder_name']= name_folder
        df["Folder_path"] = folder_path
        df["Label_name"] = label_name
        df["Image_path"] =  df['Folder_path']+ "/ImageFolder/Image_" + df[' Frame #'].str.zfill(5) + '.jpg'
        if complete_df.empty:
            complete_df = df
        else:
            complete_df = pd.concat([df, complete_df])
    return complete_df
# a=get_image_data('sipENqLc', 'p1')
# print(a.head().values.tolist())

In [0]:
#Image read/write and export the directories to csv file
start_time = time.time()
def get_all_folders_data():
    all_folder_data = pd.DataFrame()
    for key, value in map_folder_labels.items():
        folder_data = get_image_data(key, value)
        if all_folder_data.empty:
            all_folder_data = folder_data
        else:
            all_folder_data = pd.concat([folder_data, all_folder_data])
    all_folder_data.to_csv('/hadoopfs/fs1/dataiku/data_dir/managed_folders/PARTICULATEIMAGECLASSIFICATIONONSVP/hAbB1mJR/complete_csv_data.csv', index = False)
get_all_folders_data()
end_time = time.time()
elsp_time = end_time-start_time
print(elsp_time/60)

In [0]:
#multi processing
import multiprocessing
def folders_connection():
    p1 = dataiku.Folder("PIojoFlG")
    p3 = dataiku.Folder("59PiMVgQ")
    p2 = dataiku.Folder("AZvdljVp")
    p4 = dataiku.Folder("Y8jS78ta")
    p5 = dataiku.Folder("A0nzUMQ2")
    s1 = dataiku.Folder("Oay7P2WG")
    s2 = dataiku.Folder("mbeOHgg1")
    return p1,p2,p3,p4,p5,s1,s2
p1,p2,p3,p4,p5,s1,s2 = folders_connection()
def check_folder_obj(folder_name):
    if folder_name == 'p1':
        return p1
    elif folder_name == 'p2':
        return p2
    elif folder_name == 'p3':
        return p3
    elif folder_name == 'p4':
        return p4
    elif folder_name == 'p5':
        return p5
    elif folder_name == 's1':
        return s1
    elif folder_name == 's2':
        return s2

import time
st = time.time()
base_dir = '/hadoopfs/fs1/dataiku/data_dir/managed_folders/PARTICULATEIMAGECLASSIFICATIONONSVP/hAbB1mJR/Cropped_images/'
df = pd.read_csv('/hadoopfs/fs1/dataiku/data_dir/managed_folders/PARTICULATEIMAGECLASSIFICATIONONSVP/hAbB1mJR/complete_csv_data.csv', chunksize = 10000)
for i, reader in enumerate(df):
    processes = []
    reader['unq'] = reader.groupby('Image_path').cumcount().astype(str)
    for idx, row in reader.iterrows():
        p = multiprocessing.Process(target = crop_save_image, args = (row,))
        processes.append(p)
        p.start()
    for process in processes:
        process.join()
    break
et = time.time()
print((et-st)/3600)

In [0]:
def crop_save_image(row):
    image_path = row['Image_path']
    folder_obj = check_folder_obj(row['Label_name'])
    pil_image=Image.open(folder_obj.get_download_stream(image_path)) # open image using PIL
    x_left,y_top,x_right,y_bottom = int(row1['X Left']),int(row1['Y Top']),int(row1['X Right']), int(row1['Y Bottom'])
    if x_left == 0 and x_right == 0:
        x_left,x_right = 1, 2
    if y_top == 0 and y_bottom == 0:
        y_top,y_bottom = 1,2
    cropped_img = pil_image.crop((x_left,y_top,x_right,y_bottom))
    if len(unq) == 0:
        output_path = base_dir + str(row['Label_name']) + "_" + row['Folder_path']+"_Image_"+ str(row[' Frame #'])+'.jpg'
    else:
        output_path = base_dir + str(row['Label_name']) + "_" + row['Folder_path']+"_Image_"+ str(row[' Frame #'])+ "_"+str(row['unq'])+'.jpg'
    cropped_img.save(output_path)

In [0]:
all_folder_data = pd.read_csv('/hadoopfs/fs1/dataiku/data_dir/managed_folders/PARTICULATEIMAGECLASSIFICATIONONSVP/hAbB1mJR/complete_csv_data.csv')
base_dir = '/hadoopfs/fs1/dataiku/data_dir/managed_folders/PARTICULATEIMAGECLASSIFICATIONONSVP/hAbB1mJR/Cropped_images/'
folder_names = list(map_folder_labels.keys())
for f_name in folder_names:
    st = time.time()
    folder_obj = dataiku.Folder(f_name)
    print("Started folder:", f_name)
    folder_data = all_folder_data[all_folder_data['Folder_name']==f_name]
    try:
        unq_data = folder_data.drop_duplicates(subset = ['Folder_path', ' Frame #'])
        for idx, row in unq_data.iterrows():
            image_data = folder_data[(folder_data['Folder_path'] == row['Folder_path']) & (folder_data[' Frame #'] == row[' Frame #'])]
            image_data.reset_index(inplace=True)
            image_data.sort_values(by = ['Folder_path',' Frame #'], axis = 0, inplace = True)
            for idx1, row1 in image_data.iterrows():
                image_path = row['Image_path']
                pil_image=Image.open(folder_obj.get_download_stream(image_path)) # open image using PIL
                x_left,y_top,x_right,y_bottom = int(row1['X Left']),int(row1['Y Top']),int(row1['X Right']), int(row1['Y Bottom'])
                if x_left == 0 and x_right == 0:
                    x_left,x_right = 1, 2
                if y_top == 0 and y_bottom == 0:
                    y_top,y_bottom = 1,2
                cropped_img = pil_image.crop((x_left,y_top,x_right,y_bottom))
                if len(image_data) == 1:
                    output_path = base_dir + str(row['Label_name']) + "_" + row['Folder_path']+"_Image_"+ str(row[' Frame #'])+'.jpg'
                else:
                    output_path = base_dir + str(row['Label_name']) + "_" + row['Folder_path']+"_Image_"+ str(row[' Frame #'])+ "_"+str(idx1)+'.jpg'
                try:
                    cropped_img.save(output_path)
                except Exception as e:
                    print(e)
                    print(row1)
                    print(output_path)
                    print(x_left,x_right,y_top,y_bottom)
    except Exception as e1:
        print(e1)
        print(image_path)

    et = time.time()
    esp = et - st
    print("Time Elaspsed for", f_name, esp)

In [0]:
all_folder_data = pd.read_csv('/hadoopfs/fs1/dataiku/data_dir/managed_folders/PARTICULATEIMAGECLASSIFICATIONONSVP/hAbB1mJR/complete_csv_data.csv')
base_dir = '/hadoopfs/fs1/dataiku/data_dir/managed_folders/PARTICULATEIMAGECLASSIFICATIONONSVP/hAbB1mJR/Filtered_cropped_Images/'
folder_names = list(map_folder_labels.keys())
for f_name in folder_names:
    st = time.time()
    folder_obj = dataiku.Folder(f_name)
    print("Started folder:", f_name)
    folder_data = all_folder_data[all_folder_data['Folder_name']==f_name]
    try:
        unq_data = folder_data.drop_duplicates(subset = ['Folder_path', ' Frame #'])
        for idx, row in unq_data.iterrows():
            image_data = folder_data[(folder_data['Folder_path'] == row['Folder_path']) & (folder_data[' Frame #'] == row[' Frame #'])]
            image_data.reset_index(inplace=True)
            image_data.sort_values(by = ['Folder_path',' Frame #'], axis = 0, inplace = True)
            for idx1, row1 in image_data.iterrows():
                image_path = row['Image_path']
                pil_image=Image.open(folder_obj.get_download_stream(image_path)) # open image using PIL
                x_left,y_top,x_right,y_bottom = int(row1['X Left']),int(row1['Y Top']),int(row1['X Right']), int(row1['Y Bottom'])
                # Creating cropping width for edge-sided particulates
                if (x_right - x_left) < 1:
                    x_left,x_right = 1, 2
                if (y_bottom - y_top) <1:
                    y_top,y_bottom = 1,2
                total_eles = (x_right - x_left) * (y_bottom - y_top)
                # Maintaining considerbale crop size
                if total_eles > 100:
                    cropped_img = pil_image.crop((x_left,y_top,x_right,y_bottom))
                    array_data = np.asarray(cropped_img)
                    # Checking the black spot intensity
                    black_pixel_count = np.count_nonzero(array_data < 100)
                    # Save the image if contains reasonable amount of blackspots
                    if black_pixel_count:
                        if len(image_data) == 1:
                            output_path = base_dir + str(row['Label_name']) + "_" + row['Folder_path']+"_Image_"+ str(row[' Frame #'])+'.jpg'
                        else:
                            output_path = base_dir + str(row['Label_name']) + "_" + row['Folder_path']+"_Image_"+ str(row[' Frame #'])+ "_"+str(idx1)+'.jpg'
                        try:
                            cropped_img.save(output_path)
                        except Exception as e:
                            print(e)
                            print(row1)
                            print(output_path)
                            print(x_left,x_right,y_top,y_bottom)
    except Exception as e1:
        print(e1)
        print(image_path)

    et = time.time()
    esp = et - st
    print("Time Elaspsed for", f_name, esp)

In [0]:
(6137+1186+196+21+130+76+4474)/3600

In [0]:
# #test image cropping
# import matplotlib.pyplot as plt

# with (protein_compatibility_particles.get_download_stream('30 SS093 F3 Delivered R1-Run009_5200-3244_210121-230015/ImageFolder/Image_01657.jpg')) as f:
#     im = Image.open( f).convert('L')
#     cropped_img = im.crop((398,0,409,0))
#     print(cropped_img)
#     plt.imshow(cropped_img)
#     plt.show()

#     #img2.save("/hadoopfs/fs1/dataiku/data_dir/managed_folders/IMAGECLASSIFICATION/JUMGKyul/test_image.jpg")

In [0]:
# Clustering using pretrained VGG16 -Transfer Learning

reverse_map_folder_labels = {'p1' : 'PIojoFlG',
                     'p3' : '59PiMVgQ',
                     'p2' : 'AZvdljVp',
                     'p4' : 'Y8jS78ta',
                     'p5' : 'A0nzUMQ2',
                     's1' : 'Oay7P2WG',
                     's2' : 'mbeOHgg1'}




# Variables
# imdir = 'C:/indir/'
targetdir = "/hadoopfs/fs1/dataiku/data_dir/managed_folders/PARTICULATEIMAGECLASSIFICATIONONSVP/hAbB1mJR"
number_clusters = 7
df = pd.read_csv("/hadoopfs/fs1/dataiku/data_dir/managed_folders/PARTICULATEIMAGECLASSIFICATIONONSVP/hAbB1mJR/all_images.csv")
# Loop over files and get features
start_time = time.time()
featurelist = []
filelist = list(df.image_paths)
for index, row in df.iterrows():
    folder_obj = dataiku.Folder(reverse_map_folder_labels[row['label']])
    img=Image.open(folder_obj.get_download_stream(row['image_paths']))#.convert('L') # open image using PIL
    img_data = np.array(img)
    #img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)
    features = np.array(img_data)
    #features = np.array(model.predict(img_data))
    featurelist.append(features.flatten())

# Clustering
kmeans = KMeans(n_clusters=number_clusters, random_state=0).fit(np.array(featurelist))

# Copy images renamed by cluster
# Check if target dir exists
# Copy with cluster na
cluster = [m for i,m in enumerate(kmeans.labels_)]
df["Cluster_name"] = cluster

df.to_csv("/hadoopfs/fs1/dataiku/data_dir/managed_folders/PARTICULATEIMAGECLASSIFICATIONONSVP/hAbB1mJR/images_cluster.csv")
# for i, m in enumerate(kmeans.labels_):
#     print("    Copy: %s / %s" %(i, len(kmeans.labels_)), end="\r")
#     print(filelist[i], targetdir + str(m) + "_" + str(i) + ".jpg", str(m))
end_time = time.time()
elapsed_time= end_time-start_time
print('Time taken =', elapsed_time)

In [0]:
# Clustering
kmeans = KMeans(n_clusters=number_clusters, random_state=0).fit(np.array(featurelist))

# Copy images renamed by cluster
# Check if target dir exists
# Copy with cluster na
cluster = [m for i,m in enumerate(kmeans.labels_)]
df["Cluster_name"] = cluster

df.to_csv("/hadoopfs/fs1/dataiku/data_dir/managed_folders/IMAGECLASSIFICATION/JUMGKyul/images_cluster.csv")
# for i, m in enumerate(kmeans.labels_):
#     print("    Copy: %s / %s" %(i, len(kmeans.labels_)), end="\r")
#     print(filelist[i], targetdir + str(m) + "_" + str(i) + ".jpg", str(m))
end_time = time.time()
elapsed_time= end_time-start_time
print('Time taken =', elapsed_time)

In [0]:
cluster = [m for i,m in enumerate(kmeans.labels_)]
print(len(cluster))
df1 = df.head(21)
df1["Cluster_name"] = cluster

df1.to_csv("/hadoopfs/fs1/dataiku/data_dir/managed_folders/IMAGECLASSIFICATION/JUMGKyul/top_20_images_cluster.csv")